CHAPTER 19
 Migrating to SQL Server Solutions in Azure

Open Source PowerShell migration option


##Migration Process


In [0]:
Install-Module Az.Resources
Install-Module Az.Storage
Install-Module dbatools


#Migration Code
Start by populating the parameters that define the names of source/target instances and temporary Azure Blob Storage account.


In [0]:
# temporary resources needed for backups
$location = "westus"
$resourceGroup = "temp-migration-demo-rg"
$blobStorageAccount = "temp-demostorage"
$containerName = "backups"
 # source and target instances
$sourceInstance = "SOURCESQLSERVER"
$sourceDatabase = "WideWorldImporters"
 $targetInstance = "targetmi.public.920d05d7463d.database.windows.net,3342"
$targetDatabase = "WideWorldImporters"


Storage setup
In order to move your databases, you need a temporary Azure Blob Storage account. If you already have one, you can skip this step.
The following PowerShell script creates an Azure Blob Storage account that will be used in this example:



In [0]:
New-AzResourceGroup -Name $resourceGroup -Location $location
$storageAccount = New-AzStorageAccount -ResourceGroupName $resourceGroup `
-Name $blobStorageAccount `
-Location $location `
-SkuName Standard_LRS `
-Kind StorageV2
 
$ctx = $storageAccount.Context
New-AzStorageContainer -Name $containerName -Context $ctx -Permission Container


##Source instance setup

Generate the SAS key, which enables your SQL Server instance to access the Azure Blob Storage account and puts the database backups in that location. This SAS key should be stored in the credential object on the source SQL Server instance:


In [0]:
$sas = (New-AzStorageAccountSASToken -Service Blob -ResourceType Object -Permission "rw" -Context $ctx).TrimStart('?')
$sourceCred = New-DbaCredential -SqlInstance $sourceInstance `
                           -Name "https://$blobStorageAccount.blob.core.windows.net/$containerName" `
                           -Identity "SHARED ACCESS SIGNATURE" `
                           -SecurePassword (ConvertTo-SecureString $sas -AsPlainText -Force)



Database migration
Take backups of your SQL Server databases and place them in Azure Blob Storage. The simplest way is to backup directly to a URL. Here is where dbatools comes in. 



In [0]:
Backup-DbaDatabase -SqlInstance $sourceInstance -Database $sourceDatabase `
                   -AzureBaseUrl "https://$blobStorageAccount.blob.core.windows.net/$containerName" `
                   -BackupFileName "WideWorldImporters.bak" `
                   -Type Full -Checksum -CopyOnly

                   


Migrating databases to target instance
The migration stage requires you to create a SAS token that will enable managed instance to read a .bak file from Azure Blob Storage, create a credential with this SAS token, and restore the databases:


In [0]:
## Generate new SAS token that will read .bak file
$sas = (New-AzStorageAccountSASToken -Service Blob -ResourceType Object -Permission "r" -Context $ctx).TrimStart('?') # -ResourceType Container,Object
 
$targetLogin = Get-Credential -Message "Login to target Managed Instance as:"
$target = Connect-DbaInstance -SqlInstance $targetInstance -SqlCredential $targetLogin
$targetCred = New-DbaCredential -SqlInstance $target `
                           -Name "https://$blobStorageAccount.blob.core.windows.net/$containerName" `
                           -Identity "SHARED ACCESS SIGNATURE" `
                           -SecurePassword (ConvertTo-SecureString $sas -AsPlainText -Force) `
                           -Force
Restore-DbaDatabase -SqlInstance $target -Database $targetDatabase `
                   -Path "https://$blobStorageAccount.blob.core.windows.net/$containerName/WideWorldImporters.bak"


Server-level objects migration
Once you migrate your database objects, you may still need to migrate server-level objects such as Agent jobs and operators, or logins.
Dbatools provides a set of useful scripts that you can apply to migrate these objects using Copy-Dba commands.  Of the next set of commands, customize and use only those relevant to your environment. As mentioned earlier, this is where you can migrate Agent jobs, linked servers, and end points.  If you do not see the command you need here check the documentation at: https://docs.dbatools.io.


In [0]:
Copy-DbaSysDbUserObject -Source $sourceInstance -Destination $targetInstance -DestinationSqlCredential $targetLogin
Copy-DbaDbMail -Source $sourceInstance -Destination $targetInstance -DestinationSqlCredential $targetLogin
Copy-DbaAgentOperator -Source $sourceInstance -Destination $targetInstance -DestinationSqlCredential $targetLogin
Copy-DbaAgentJobCategory -Source $sourceInstance -Destination $targetInstance -DestinationSqlCredential $targetLogin
Copy-DbaAgentJob -Source $sourceInstance -Destination $targetInstance -DestinationSqlCredential $targetLogin
Copy-DbaAgentSchedule -Source $sourceInstance -Destination $targetInstance -DestinationSqlCredential $targetLogin
Copy-DbaLogin -Source $sourceInstance -Destination $targetInstance -DestinationSqlCredential $targetLogin -ExcludeSystemLogins
Copy-DbaLinkedServer -Source $sourceInstance -Destination $targetInstance -DestinationSqlCredential $targetLogin -LinkedServer] <Object[]>
Copy-DbaEndpoint -Source $sourceInstance -Destination $targetInstance -DestinationSqlCredential $targetLogin [-Endpoint] <Object[]>]
 


Cleanup process for PoSh migration
If your Azure Blob Storage account is temporary storage that you need to use when moving databases, you can remove these resources using the following PowerShell command.  It is good practice to always clean up resources you no longer need.


In [0]:
 Remove-AzResourceGroup -Name $resourceGroup -Force